<a href="https://colab.research.google.com/github/Shumookh/Best-selling-product/blob/main/Best_selling_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

هل شعرت يومًا بالقلق من عدم بيع منتجاتك؟ تخيل أنك تمتلك متجرًا صغيرًا، وكل شهر يأتي ومعه تساؤلات: "ما هو المنتج الأكثر مبيعاً لشهر القادم؟"
 فقد أصبح هذا واقع ممكن مع استخدام تقنية تُدعى "الشبكات العصبية". حيث أن بيانات المبيعات السابقة، مثل أنواع المنتجات، والشهور، وأيام العطلات. لن تشعر بالقلق بعد الآن، بل ستحصل على الوقت لتحسين متجرك وخدمة عملائك بشكل أفضل. تذكر: الحلول الذكية يمكن أن تحول التحديات إلى نجاحات

In [37]:
#------(Importing libraries)------#
#------(استدعاء المكتبات )------#
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score

#------(Reading the data)------#
#------( قراءة البيانات)------#
data = pd.read_csv('/content/dataset.csv')
data = data.dropna()

#------(Converting months to numbers)------#
#------(تحويل الشهور إلى أرقام)------#
data['Month'] = data['Month'].map({
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12})

#------(Separating features from the target variable)------#
#------(تحديد الميزات عن المتغير المستهدف)------#
X = data[['Product', 'Month', 'Special Days', 'Shelf Life']]
y = data['Number of Products Sold']

#------(Converting categorical features to numerical features)------#
#------(تحويل الميزات النوعية إلى ميزات رقمية)------#
X = pd.get_dummies(X, columns=['Product'], drop_first=True)

#------(Splitting the data into training and testing sets)------#
#------(تقسيم البيانات إلى مجموعة تدريب واختبار)------#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#------(Normalizing the data)------#
#------(تطبيع البيانات)------#
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#------(Building the neural network model)------#
#------(بناء نموذج الشبكة العصبية)------#
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # طبقة الإخراج

#------(Compiling the neural network)------#
#------(تحميع الشبكة العصبية)------#
model.compile(optimizer='adam', loss='mean_squared_error')

#------(Training the neural network)------#
#------(تدريب الشبكة العصبية)------#
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1)

#------(Making predictions)------#
#------(إجراء التنبؤات)------#
y_pred = model.predict(X_test)

#------(Preparing new data for January with all products)------#
#------(إعداد بيانات جديدة لشهر يناير مع جميع المنتجات)------#
new_data = pd.DataFrame({
    'Month': [1] * (X.shape[1] - 3),
    'Special Days': [1] * (X.shape[1] - 3),
    'Shelf Life': [24] * (X.shape[1] - 3)})

#------(Adding product columns)------#
#------( إضافة أعمدة للمنتجات)------#
for col in X.columns:
    if col.startswith('Product_'):
        new_data[col] = 0

#------(Making predictions for each product)------#
#------(إجراء التنبؤ لكل منتج)------#
predictions = {}
for product in new_data.columns:
    if product.startswith('Product_'):
        new_data[product] = 1  # تعيين المنتج الحالي
        scaled_data = scaler.transform(new_data)  # تطبيع البيانات الجديدة
        predicted_sales = model.predict(scaled_data)
        predictions[product] = predicted_sales[0][0]  # تخزين النتيجة
        new_data[product] = 0  # إعادة تعيين المنتج

#------(Identifying the best-selling product)------#
#------(تحديد المنتج الأكثر مبيعًا)------#
best_selling_product = max(predictions, key=predictions.get)

#------(Calculating R^2)------#
#------(حساب R^2)------#
r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

#------(Displaying the predicted best-selling product)------#
#------( اظهار المنتج الأكثر مبيعًا)------#
print(f"المنتج المتوقع الأكثر مبيعًا في شهر يناير القادم هو: {best_selling_product.replace('Product_', '')}")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1426939.1250
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1664602.0000 
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1854177.1250 
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1927091.5000 
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1621990.0000 
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1480299.2500 
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1645866.2500 
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1634869.8750 
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1471006.6250 
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1230542.1250
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1210248.1250
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 932963.3125 
Epoch 13/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 770694.6875 
Epoch 14/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 544